In [19]:
import pandas as pd 
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt 
import seaborn as sns
from pandas_datareader import data
import pandas_datareader as pdr
from pathlib import Path 
from scipy import stats

import dash
import dash_core_components as dcc
import dash_html_components as html

import streamlit as st

'''
Data Frames created:
broker_file = data from broker with [tickers, shares, last, close, mktvalue]
price_df = daily ytd prices of all positions 
daily_percent_change = daily percent change of all tickers
ticker_position= ticker and shares for each position
weightings = weights of all positions in the portfolio based on market value
'''


'\nData Frames created:\nbroker_file = data from broker with [tickers, shares, last, close, mktvalue]\nprice_df = daily ytd prices of all positions \ndaily_percent_change = daily percent change of all tickers\nticker_position= ticker and shares for each position\nweightings = weights of all positions in the portfolio based on market value\n'

In [2]:
cash= 26572.84
broker_file =pd.read_csv(Path('shf_positions.csv'))
daily_balance = pd.read_csv(Path('daily_balance.csv'))

daily_balance['date'] = pd.to_datetime(daily_balance['date'])

daily_balance.replace(',','', regex=True, inplace=True)
daily_balance['aum'] = daily_balance['aum'].astype(float)
daily_balance.info()
display(broker_file.head())
display(daily_balance.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    230 non-null    datetime64[ns]
 1   aum     230 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.7 KB


,Symbol,Description,Qty,Last,Close,Mkt value,Gain ($),Gain (%),Cost/share,P/E
0,CCJ,Cameco Corporation Common Stock,500,25.29,26.64,"12,645.00","9,118.69",258.59%,7.05262,--
1,ARKK,ARK Innovation ETF,70,113.24,113.82,"7,936.60","5,342.32",205.93%,37.06114,--
2,ARKG,ARK Genomic Revolution ETF,85,68.53,69.12,"5,825.90","3,400.63",140.22%,28.53259,--
3,XLK,SPDR Select Sector Fund - Technology,420,171.31,170.07,"71,950.20","35,672.07",98.33%,86.37650,--
4,DG,Dollar General Corporation Common Stock,160,224.55,222.34,"35,928.00","17,194.17",91.78%,117.08644,21.30X


,date,aum
0,2021-01-01,676001.97
1,2021-01-04,668975.07
2,2021-01-05,670881.32
3,2021-01-06,676514.02
4,2021-01-07,683265.92


In [3]:
#cleaning file --getting rid of commas in str and converting to floats instead of objects
broker_file.replace(',','', regex=True, inplace=True)
broker_file['Mkt value'] =broker_file['Mkt value'].astype(float)
aum = broker_file['Mkt value'].sum() + cash 
broker_file.head()

,Symbol,Description,Qty,Last,Close,Mkt value,Gain ($),Gain (%),Cost/share,P/E
0,CCJ,Cameco Corporation Common Stock,500,25.29,26.64,12645.0,9118.69,258.59%,7.05262,--
1,ARKK,ARK Innovation ETF,70,113.24,113.82,7936.6,5342.32,205.93%,37.06114,--
2,ARKG,ARK Genomic Revolution ETF,85,68.53,69.12,5825.9,3400.63,140.22%,28.53259,--
3,XLK,SPDR Select Sector Fund - Technology,420,171.31,170.07,71950.2,35672.07,98.33%,86.37650,--
4,DG,Dollar General Corporation Common Stock,160,224.55,222.34,35928.0,17194.17,91.78%,117.08644,21.30X


In [4]:
broker_file= broker_file.replace('BRK.B', 'BRK-B')
tickers  = broker_file['Symbol'].tolist()
weights = []

for i in broker_file:
    weight = broker_file['Mkt value']/aum
    weights.append(weight)
    break


ticker_weights = list(zip(tickers, weights))


2021-11-21 09:16:08.724 INFO    numexpr.utils: Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-11-21 09:16:08.724 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [5]:
# creating ticker list 
#importing data
symbol= tickers
start='2020-11-20'
end='2021-11-20'

price_df=data.get_data_yahoo(symbol, start, end)['Close']
price_df.head()


Symbols,CCJ,ARKK,ARKG,XLK,DG,NKE,FB,INFO,BA,SPY,...,NTAP,EA,XLP,DGS,C,XLU,VZ,T,PAA,SND
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-20,9.95,104.900002,76.449997,120.110001,210.229996,132.979996,269.700012,92.190002,199.619995,355.329987,...,51.889999,121.870003,66.500000,45.310001,51.650002,63.540001,60.040001,28.320000,7.93,1.78
2020-11-23,9.92,107.419998,77.059998,120.080002,211.039993,134.130005,268.429993,93.660004,211.529999,357.459991,...,53.560001,121.860001,66.519997,45.419998,53.299999,63.549999,60.230000,28.620001,8.51,1.92
2020-11-24,10.34,106.139999,74.720001,121.750000,212.979996,134.699997,276.920013,93.620003,218.490005,363.220001,...,55.360001,120.250000,66.970001,45.799999,57.060001,64.230003,60.639999,29.230000,8.66,1.90
2020-11-25,10.15,108.480003,74.730003,122.010002,217.000000,135.539993,275.589996,92.300003,217.610001,362.660004,...,53.520000,121.510002,66.930000,45.669998,57.060001,64.400002,60.410000,28.990000,8.63,2.05
2020-11-27,10.13,111.779999,79.610001,122.660004,218.009995,134.250000,277.809998,92.580002,216.500000,363.670013,...,53.259998,124.169998,66.919998,46.150002,56.669998,63.759998,60.580002,29.030001,8.28,1.97


In [6]:
#creating daily percent change df 

daily_percent_change = price_df.pct_change()
daily_percent_change = daily_percent_change[1:]
daily_percent_change.head()

Symbols,CCJ,ARKK,ARKG,XLK,DG,NKE,FB,INFO,BA,SPY,...,NTAP,EA,XLP,DGS,C,XLU,VZ,T,PAA,SND
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-23,-0.003015,0.024023,0.007979,-0.000250,0.003853,0.008648,-0.004709,0.015945,0.059663,0.005994,...,0.032184,-0.000082,0.000301,0.002428,0.031946,0.000157,0.003165,0.010593,0.073140,0.078652
2020-11-24,0.042339,-0.011916,-0.030366,0.013907,0.009193,0.004250,0.031628,-0.000427,0.032903,0.016114,...,0.033607,-0.013212,0.006765,0.008366,0.070544,0.010700,0.006807,0.021314,0.017626,-0.010417
2020-11-25,-0.018375,0.022046,0.000134,0.002136,0.018875,0.006236,-0.004803,-0.014100,-0.004028,-0.001542,...,-0.033237,0.010478,-0.000597,-0.002838,0.000000,0.002647,-0.003793,-0.008211,-0.003464,0.078947
2020-11-27,-0.001970,0.030420,0.065302,0.005327,0.004654,-0.009517,0.008055,0.003034,-0.005101,0.002785,...,-0.004858,0.021891,-0.000149,0.010510,-0.006835,-0.009938,0.002814,0.001380,-0.040556,-0.039024
2020-11-30,-0.010859,0.005278,0.026253,0.006685,0.002615,0.003352,-0.003024,0.074314,-0.026744,-0.004427,...,0.000939,0.028831,-0.000149,-0.009317,-0.028234,-0.014429,-0.002806,-0.009645,-0.041063,-0.012690


In [7]:
ticker_position = broker_file[['Symbol','Qty', 'Mkt value']]
ticker_position.head()

,Symbol,Qty,Mkt value
0,CCJ,500,12645.0
1,ARKK,70,7936.6
2,ARKG,85,5825.9
3,XLK,420,71950.2
4,DG,160,35928.0


In [8]:
correlations  = daily_percent_change.corr(method='pearson')
correlations.head()

Symbols,CCJ,ARKK,ARKG,XLK,DG,NKE,FB,INFO,BA,SPY,...,NTAP,EA,XLP,DGS,C,XLU,VZ,T,PAA,SND
Symbols,,,,,,,,,,,,,,,,,,,,,
CCJ,1.000000,0.269514,0.287092,0.203653,-0.004605,0.152984,0.125291,0.042924,0.310685,0.274504,...,0.159359,0.132965,0.062260,0.373846,0.265582,-0.034141,-0.073547,0.061407,0.294142,0.345318
ARKK,0.269514,1.000000,0.918589,0.622898,0.215068,0.262515,0.363304,0.147482,0.329445,0.498136,...,0.257678,0.317955,-0.027586,0.459539,0.127016,-0.011170,-0.243844,-0.138110,0.179553,0.198090
ARKG,0.287092,0.918589,1.000000,0.540360,0.183542,0.275094,0.293774,0.162753,0.311036,0.446595,...,0.248014,0.277584,-0.020435,0.404324,0.123219,-0.003617,-0.184761,-0.124386,0.202716,0.210409
XLK,0.203653,0.622898,0.540360,1.000000,0.352576,0.356333,0.584538,0.446470,0.291087,0.871594,...,0.402451,0.362153,0.395275,0.510678,0.163102,0.232687,-0.000156,0.010526,0.173545,0.114219
DG,-0.004605,0.215068,0.183542,0.352576,1.000000,0.150935,0.227629,0.264370,0.085726,0.381391,...,0.155966,0.084842,0.370416,0.156288,0.048036,0.341447,0.148291,0.087901,0.051738,0.127334


In [9]:
correlations.hvplot.heatmap(
    title = 'Heatmap of South Hill Equity Fund Single Name Correlations',
    height=700, width = 1000, 
    colorbar=False, 
    rot = 60

)

:HeatMap   [columns,index]   (value)

In [68]:
#Creating return statsitics for different tmie periods for Beta regression

returns = daily_percent_change
returns.dropna(inplace=True)

for i in returns:
    y_3m = returns[-20:]
    x_3m = returns[-20:]['SPY']
    y_6m= returns[-180:]
    x_6m = returns[-180:]['SPY']
    y_1y=returns[-252:]
    x_1y = returns[-252:]['SPY']            

'''
shf_betas = pd.DataFrame()
shf_betas['3m_Betas']: x_3m
shf_betas['6m_Betas']: x_6m
shf_betas['1yr_Beta']: x_1y
'''

x_6m.head()

Date
2021-03-09    0.014278
2021-03-10    0.006225
2021-03-11    0.010139
2021-03-12    0.001347
2021-03-15    0.005964
Name: SPY, dtype: float64

In [100]:
#loopong over df in order to regress each SN 

import statsmodels.api as sm

for columns in y_3m:
    Y = y_3m[columns]
    X = x_3m
    #X = sm.add_constant(X)
    model = sm.OLS(X,Y)
    results_3m = model.fit()
    results_3m_params=results_3m.params
    
    
for columns in y_6m:
    Y = y_6m[columns]
    X = x_6m
    #X = sm.add_constant(X)
    model = sm.OLS(X,Y)
    results_6m = model.fit()
     #print(results_6m.params))
    
    
def results_summary_to_datafram_3m(results_3m_params):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    #conf_lower = results.conf_int()[0]
    #conf_higher = results.conf_int()[1]

    results_df_3m = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               #"conf_lower":conf_lower,
                               #"conf_higher":conf_higher
                                })

    #Reordering...
    results_df_3m = results_df_3m[["coeff","pvals"]]
    return results_df_3m




In [101]:
results_summary_to_datafram_3m(results_3m_params)

,coeff,pvals
SND,0.050551,0.204126


In [ ]:
shf_betas = pd.DataFrame()
shf_betas['Ticker']=ticker_position['Symbol']
shf_betas['3m_Betas']= results_3m
shf_betas['6m_Betas']= results_6m
#shf_betas['1yr_Beta']: x_1y 

shf_betas.head()

In [55]:



shf_betas.head()

""


In [13]:
#Calcualting 3m, 6m and 1yr betas (all daily)

shf_betas = []

#Slicing percent change SN table for 3m, 6m and 1yr increments 
daily_percent_change_3m  = daily_percent_change[:60]
daily_percent_change_6m  = daily_percent_change[:180]
daily_percent_change_1yr  = daily_percent_change[:252]


#Slicing percent change Market (SPY) table for 3m, 6m and 1yr increments 
spx_daily_percent_change_3m  = daily_percent_change[:60]['SPY']                         
spx_daily_percent_change_6m  = daily_percent_change[:180]['SPY']
spx_daily_percent_change_1yr  = daily_percent_change[:252]['SPY']

#Calulating Market Variance for different time periods
market_variance_3m= spx_daily_percent_change_3m.var()
market_variance_6m= spx_daily_percent_change_6m.var()
market_variance_1yr= spx_daily_percent_change_1yr.var()

#Calcualting Covariances for differnt time periods 
covariance_3m = daily_percent_change_3m.cov(market_variance_3m)
covariance_6m = daily_percent_change_6m.cov(market_variance_6m)
covariance_1yr = daily_percent_change_1yr.cov(market_variance_1yr)
    
    
#Calcualting Beta for difference time periods

beta_3m = covariance_3m/market_variance_3m
beta_6m = covariance_6m/market_variance_6m
beta_1yr = covariance_1yr/market_variance_1yr    


#Creating SHF Table

for i in 

shf_betas = pd.DataFrame()
shf_betas['3m_Betas']: beta_3m
shf_betas['6m_Betas']: beta_6m
shf_betas['1yr_Beta']: beta_1yr


'''
shf_betas = shf_betas.join(pd.DataFrame(
    {
        '3m_Betas': beta_3m,
        '6m_Betas': beta_6m,
        '1yr_Beta': beta_1yr
    }, index=shf_betas.index
))

'''





#shf_betas = list(zip(tickers, beta_table))

#shf_betas = pd.DataFrame(shf_betas)
#shf_betas.columns = ['Ticker', '3m Beta']

shf_betas.head()



""


In [ ]:
#Calcualting Beta table for 3m, 6m and ytd betas for all single names in portfolio 
#daily_percent_change.head
variance= daily_percent_change['SPY'].var()
beta_table = []




for i in daily_percent_change:
    covariance = daily_percent_change[:60][i].cov(daily_percent_change[:60]['SPY'])
    beta = covariance/variance 
    beta_table.append(beta)

shf_betas = list(zip(tickers, beta_table))

shf_betas = pd.DataFrame(shf_beta)
shf_betas.columns = ['Ticker', '1yr Beta']

shf_betas


In [ ]:
#Portfolio returns

#daily_balance = daily_balance.set_index('date')

portfolio_daily_returns = daily_balance.pct_change()
portfolio_daily_returns.rename(columns ={'aum':'returns'})
cumulative_portfolio_returns =(1 + portfolio_daily_returns).cumprod() - 1

cumulative_portfolio_returns.plot(figsize=(12,6), title ='South Hill Equity Fund YTD Portfolio Returns')

In [ ]:
#Rolling beta and Std Deviations 

#annualized_shf_std
portfolio_daily_returns.rolling(window=30).mean().plot(figsize=(12,6), title = 'SHF Rolling 30d Volatility');

In [ ]:
#sharpe ratios of individual names 
average_annual_return = daily_percent_change.mean()*252
annualized_std_deviaton = daily_percent_change.std()*252**(1/2)
sharpe_ratios= average_annual_return/annualized_std_deviaton

sharpe_ratios.plot.bar(figsize=(20,10),title = 'Single Name Sharpe Ratios', rot =70);


In [ ]:
daily_percent_change.plot.box(figsize = (20,10), title = 'Box plot for each single name postion in the SHF Portfolio');

In [ ]:
cumulative_returns_sn = (1+daily_percent_change).cumprod()-1
cumulative_returns_sn.hvplot.line(figsize= (20,10), title='YTD Cumulative Returns')

In [ ]:
daily_percent_change.hvplot.line(figsize =(20,10))

In [ ]:
daily_balance.hvplot.line(title = 'Daily AUM for 2021')